# Python tSNE implimentation

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize                # the SciPy function minimizers
from sklearn.manifold import TSNE    # scikit-learn's t-SNE implementation
import idx2numpy
import time
%matplotlib notebook

In [10]:
def calc_pji(D, sigmas):
    M   = D.shape[0]           # M = number of rows, samples
    pji = np.zeros((M,M))
    for i in range(M):
        for j in range(M):     # it's not symmetric, because of sigma_i
            pji[i,j] = np.exp(-D[i,j]**2 / (2 * sigmas[i]**2)) if j != i else 0.0
        Z = np.sum(pji[i])
        pji[i] = pji[i] / Z
    return pji    

In [11]:
def calc_D(X):
    M = X.shape[0]       # ncells
    D = np.zeros((M,M))
    for i in range(M):
        for j in range(0,i):
            D[i,j] = np.linalg.norm(X[i] - X[j])
            D[j,i] = D[i,j]
    return D

In [16]:
def calc_perplexity_diff(sigmai, Di, target_perplexity):   # <Di> is one row of the distance matrix.
    pji = np.exp( -Di**2 / (2 * sigmai**2))
    Z   = np.sum(pji)
    pji = pji / Z

    H = 0                     # the Shannon entropy calculation for the p_j|i given this sigma_i value
    for j in range(len(Di)):
        if pji[j] > 0:
            H -= pji[j] * np.log2(pji[j])

    perplexity = 2**H
    return perplexity - target_perplexity

In [18]:
def calc_sigmas(D, target_perplexity):
    M     = D.shape[0]
    sigma = np.zeros(M)
    for i in range(M):
        a, b = 1.0, 1.0   # Start testing a,b at 1.0
        while calc_perplexity_diff(a, D[i], target_perplexity) >= 0: a /= 2    # Move a in 0.5x steps until f(a) < 0
        while calc_perplexity_diff(b, D[i], target_perplexity) <= 0: b *= 2    #  ... b in 2x steps until f(a) > 0
        sigma[i] = scipy.optimize.bisect(calc_perplexity_diff, a, b, args=(D[i],target_perplexity))
    return sigma

In [19]:
def calc_P(X, target_perplexity):
    D      = calc_D(X)
    sigmas = calc_sigmas(D, target_perplexity)
    pji    = calc_pji(D, sigmas)
    P      = (pji + pji.T) / (2 * X.shape[0])
    return P

In [20]:
def calc_Q(Y):
    M = Y.shape[0]
    Q = np.zeros((M,M))
    Z = 0.
    for i in range(M):
        for j in range(i):  # This nested for loop leaves q_ii at zero.
            Q[i,j] = 1.0 / (1 + np.linalg.norm( Y[i] - Y[j] )**2)   # the t distribution, or Cauchy
            Q[j,i] = Q[i,j]
            Z += 2*Q[i,j]
    Q = Q / Z
    return Q

In [21]:
def KL_dist(Y, P):
    M      = P.shape[0]
    L      = len(Y) // M
    Y      = np.reshape(Y, (M,L))    # scipy treats Y is a vector; reshape it to an Mx2 matrix
    Q      = calc_Q(Y)
    kldist = 0.
    grad   = np.zeros((M, L))        # from our perspective, the gradient is an Mx2 matrix

    # The objective function is KL(P||Q)
    for i in range(M):
        for j in range(M):
            if P[i,j] > 0:
                kldist += P[i,j] * np.log( P[i,j] / Q[i,j] )

    # The gradient is defined on p. 2584 of van der Maaten (2008);
    # the derivation is in their appendix.
    for i in range(M):
        for j in range(M):
            grad[i] += (P[i,j] - Q[i,j]) * (Y[i] - Y[j]) * (1.0 / (1 + np.linalg.norm( Y[i] - Y[j] )**2))
        grad[i] *= 4.0

    return kldist, grad.flatten()    # .flatten(), because SciPy wants to see that gradient just as a vector.


In [22]:
def my_tsne(X, target_perplexity):
    P = calc_P(X, target_perplexity)
    Y = np.random.normal(0., 1e-4, (X.shape[0], 2))   # random initial Y is drawn Normal(0, 1e-4 I)
    
    result = scipy.optimize.minimize(KL_dist, Y.flatten(), args=(P), jac=True)
    # Explication of what we pass to the minimizer:
    #  KL_dist is the objective function.
    #  Y.flatten() is our initial point Y, flattened to a single vector for SciPy's consumption.
    #  args=(P) is the interface for handing SciPy a bundle of constant additional data that it needs to pass to the
    #           objective function; here, the stochastic neighbor distribution in the original space that we're 
    #           trying to match
    # jac=True is the poorly-documented way of telling SciPy that our objective function is going to return both
    #          the f(Y) value and the f'(Y) gradient. "jac" means Jacobian matrix. The other way to do this
    #          is to pass the name of a function that calculates the gradient, i.e. jac=my_gradient_func
    # 
    # When the minimizer returns, it passes a tuple of information back. 
    # The parts we care about are:
    #    result.x        The optimized solution -- our embedded Y coordinates, as a Mx2-long vector that we'll reshape.
    #    result.success  True if the minimizer succeeded. We ought to check this.
    #    result.fun      The objective function value at the solution. 
    #    result.nit      Number of iterations it took

    Y = result.x.reshape(X.shape[0], 2)
    print (result.success)
    print (result.nit)
    
    return Y, result.fun   # Return the Y points embedded by t-SNE; and the KL distance. 

In [26]:
def pca(X=np.array([]), no_dims=50):
    """
        Runs PCA on the NxD array X in order to reduce its dimensionality to
        no_dims dimensions.
    """
    (n, d) = X.shape

    X = X - np.tile(np.mean(X, 0), (n, 1))
    
    (l, M) = np.linalg.eig(np.dot(X.T, X))
    Y = np.dot(np.dot(X, M[:, 0:no_dims]),M[:, 0:no_dims].T)
    return Y



In [36]:
start_time = time.time()
X = idx2numpy.convert_from_file('../data/t10k-images.idx3-ubyte')[0:2500,:,:]
M, h, w = X.shape
N = h*w
X = X.reshape((M,N))
X = X/255.0
X = pca(X, 50).real
target_perplexity = 5.0
Y, KLdist = my_tsne(X, target_perplexity)
print("--- %s seconds ---" % (time.time() - start_time))
KLdist

Preprocessing the data using PCA...
True
0
--- 589.6153552532196 seconds ---


3.7687098751895793

In [37]:
start_time = time.time()
X = idx2numpy.convert_from_file('../data/t10k-images.idx3-ubyte')[0:100,:,:]
M, h, w = X.shape
N = h*w
X = X.reshape((M,N))
X = X/255.0
X = pca(X, 50).real
target_perplexity = 5.0
Y, KLdist = my_tsne(X, target_perplexity)
print("For 100 --- %s seconds ---" % (time.time() - start_time))

Preprocessing the data using PCA...
True
0
For 100 --- 1.242521047592163 seconds ---


In [38]:
start_time = time.time()
X = idx2numpy.convert_from_file('../data/t10k-images.idx3-ubyte')[0:400,:,:]
M, h, w = X.shape
N = h*w
X = X.reshape((M,N))
X = X/255.0
X = pca(X, 50).real
target_perplexity = 5.0
Y, KLdist = my_tsne(X, target_perplexity)
print("For 400 --- %s seconds ---" % (time.time() - start_time))



Preprocessing the data using PCA...
True
0
For 400 --- 15.477382898330688 seconds ---


In [39]:
start_time = time.time()
X = idx2numpy.convert_from_file('../data/t10k-images.idx3-ubyte')[0:1000,:,:]
M, h, w = X.shape
N = h*w
X = X.reshape((M,N))
X = X/255.0
X = pca(X, 50).real
target_perplexity = 5.0
Y, KLdist = my_tsne(X, target_perplexity)
print("For 1000 --- %s seconds ---" % (time.time() - start_time))


Preprocessing the data using PCA...
True
0
For 1000 --- 94.91009283065796 seconds ---


In [40]:
start_time = time.time()
X = idx2numpy.convert_from_file('../data/t10k-images.idx3-ubyte')[0:4000,:,:]
M, h, w = X.shape
N = h*w
X = X.reshape((M,N))
X = X/255.0
X = pca(X, 50).real
target_perplexity = 5.0
Y, KLdist = my_tsne(X, target_perplexity)
print("For 4000 --- %s seconds ---" % (time.time() - start_time))


Preprocessing the data using PCA...
True
0
For 4000 --- 1491.2436559200287 seconds ---


In [41]:
start_time = time.time()
X = idx2numpy.convert_from_file('../data/t10k-images.idx3-ubyte')[0:8000,:,:]
M, h, w = X.shape
N = h*w
X = X.reshape((M,N))
X = X/255.0
X = pca(X, 50).real
target_perplexity = 5.0
Y, KLdist = my_tsne(X, target_perplexity)
print("For 8000 --- %s seconds ---" % (time.time() - start_time))


Preprocessing the data using PCA...
True
0
For 8000 --- 5938.582994937897 seconds ---


In [46]:
sample_size=[100,400,1000,4000,8000]
runtimes=[1.242521047592163,15.477382898330688,94.91009283065796,1491.2436559200287,5938.582994937897]
plt.plot(sample_size,runtimes)
plt.xlabel('M (sample size)')
plt.ylabel('Runtime (s)')
plt.title('Python runtime for t-SNE against problem size')
plt.show()